### This is used for extracting only amharic comments from an Ethiopian Music Clip called **ወንድ ልጅ ቆረጠ** by a famous Ethiopian Traditional musician named **Dagne Walle. 

In [ ]:
import os
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# Replace with your actual API key
api_key = "   "  # Make sure to insert your actual API key

# Replace with the ID of the YouTube video you want to scrape comments from if you want other video.
video_id = "UwOzBq8snm8"

def clean_text_for_excel(text):
    """Removes or replaces characters that can cause issues in Excel."""
    if isinstance(text, str):
        # Remove characters that are not valid in XML
        text = re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
        return text
    return text

def is_amharic(text):
    """
    Checks if a given text contains Amharic characters.
    This is a basic check and might not be perfect, but it's a good starting point.
    """
    if isinstance(text, str):
        return bool(re.search(r'[\u1200-\u137f]', text))  # Check for Ethiopic script
    return False

def get_comments_for_excel(youtube, video_id, max_results=100):
    """
    Retrieves comments, filters for Amharic, and cleans the text for Excel compatibility.
    """
    comments_data = []
    comment_id_counter = 1
    try:
        request = youtube.commentThreads().list(
            part="snippet,replies",
            videoId=video_id,
            maxResults=max_results,
            textFormat="plainText"
        )
        response = request.execute()

        while response:
            for item in response['items']:
                snippet = item['snippet']['topLevelComment']['snippet']
                comment_text = snippet['textDisplay']
                # Check if the comment is Amharic before processing
                if is_amharic(comment_text):
                    comment_id = f"C{comment_id_counter}"
                    author = snippet['authorDisplayName']
                    timestamp = datetime.strptime(snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ")
                    time_formatted = timestamp.strftime("%Y-%m-%dT%H:%M:%S")
                    comment_text = clean_text_for_excel(comment_text)  # Clean the text
                    likes = snippet['likeCount']
                    reply_count = item['snippet'].get('totalReplyCount', 0)

                    comments_data.append({
                        'Comment ID': comment_id,
                        'Author': author,
                        'Time': time_formatted,
                        'Comment Text': comment_text,
                        'Likes': likes,
                        'Reply Count': reply_count
                    })
                    comment_id_counter += 1

            if 'nextPageToken' in response:
                request = youtube.commentThreads().list(
                    part="snippet,replies",
                    videoId=video_id,
                    maxResults=max_results,
                    pageToken=response['nextPageToken'],
                    textFormat="plainText"
                )
                response = request.execute()
            else:
                break
    except Exception as e:
        print(f"An error occurred: {e}")
    return comments_data

if __name__ == '__main__':
    # Build the YouTube API service object
    youtube = build('youtube', 'v3', developerKey=api_key)

    # Get the comments
    comments = get_comments_for_excel(youtube, video_id, max_results=100)

    if comments:
        # Create a Pandas DataFrame
        df = pd.DataFrame(comments)

        # Save the DataFrame to an Excel file
        excel_filename = f"amharic_youtube_comments_{video_id}.xlsx"
        try:
            df.to_excel(excel_filename, index=False)
            print(f"Successfully scraped {len(comments)} Amharic comments and saved to {excel_filename}")
            print("\nFirst few rows of the output DataFrame:")
            print(df.head())
        except Exception as e:
            print(f"An error occurred while saving to Excel: {e}")
    else:
        print(f"No Amharic comments found for video ID: {video_id}")


Successfully scraped 2867 Amharic comments and saved to amharic_youtube_comments_UwOzBq8snm8.xlsx

First few rows of the output DataFrame:
  Comment ID               Author                 Time  \
0         C1           @dagnewale  2024-10-31T16:13:18   
1         C2           @dawit7411  2025-05-04T17:22:55   
2         C3  @MelakuTeferi-zl9qs  2025-05-04T12:51:20   
3         C4     @andualembazezew  2025-05-04T07:55:11   
4         C5     @andualembazezew  2025-05-04T07:54:48   

                                        Comment Text  Likes  Reply Count  
0  ከልብ አመሰግናለሁ 🙏 አስር ሚሊየን እይታ አግኝተናል ይሄደግሞ የሆነው በ...   2293          218  
1   ጀግናችን በጣም ምርጥ ስራ ነው እናመሰግናለን ❤❤🙏🙏\n12M view 😍❤❤❤      1            0  
2                ኡፍፍፍፍፍፍ የወኔ ነዳጅ የሆነ ታሪካዊ ሙዚቃ።💪💪💪💪💪💪      0            0  
3  ትክክለኛና በሰዉ ልጅ አምሮ ያለ እዉነት ነዉ ። በተለይ በዚህ ወቅት ለአ...      0            0  
4  ትክክለኛና በሰዉ ልጅ አምሮ ያለ እዉነት ነዉ ። በተለይ በዚህ ወቅት ለአ...      0            0  
